In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")

In [6]:
# 1
len(df.columns)

19

In [10]:
df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60)

In [12]:
# 2
df["duration"].std()

42.59435124195458

In [13]:
len(df)

3066766

In [14]:
df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

In [15]:
len(df)

3009173

In [16]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_5783/3500858954.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [17]:
train_dicts = df[categorical + numerical].to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = "duration"
y_train = df[target].values

In [22]:
# 4
X_train.shape

(3009173, 516)